# Summarize microbiome data by genus

The goal of this notebook is to summarize microbiome data according to host taxonomy. 
This includes several measures of richness and evenness, as well as the most abundant microbial group at several levels of taxonomy. These are then merged with the disease data to produce a final trait table.

## Import necessary libraries

First we'll import needed libraries, taking some care to accomodate older versions of QIIME2


In [54]:
from qiime2 import Artifact
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.taxa.methods import filter_table,collapse

#The below try/except block is unsightly but the alpha function got moved between recent versions of QIIME2
#and it's nice if the notebook is compatible with either
try:
    from qiime2.plugins.diversity.methods import alpha
    from qiime2.plugins.diversity.methods import alpha_phylogenetic
except:
    from qiime2.plugins.diversity.pipelines import alpha
    from qiime2.plugins.diversity.pipelines import alpha_phylogenetic
    
from qiime2.plugins.diversity.visualizers import alpha_group_significance

from qiime2.plugins.feature_table.methods import rarefy
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.methods import filter_features

from qiime2.metadata import Metadata
from os.path import abspath,exists,join
from os import mkdir
import shutil

from IPython.core.display import HTML

import pandas as pd
import numpy as np

from collections import defaultdict

## (Optional) Check what files are in the input directory

It can be useful to manually confirm that all the files are where we expect before we begin.

In [2]:
from os import listdir
listdir("../input")

['GCMP_EMP_map_r29.txt',
 'silva_metaxa2_reference_taxonomy.qza',
 'physeq_taxonomy.txt',
 'feature_table_decon_skeleton_1000.qza',
 'disease_by_genus.csv',
 'huang_roy_molecular.newick',
 'physeq_metadata.txt',
 'insertion-tree_silva_GCMP.qza',
 'feature_table_decon_tissue_1000.qza',
 'GCMP_EMP_map_r28_no_empty_samples.txt',
 'merged_disease_table.tsv',
 'feature_table_decon_all_1000.qza',
 'physeq_rooted_tree.qza',
 'huang_roy_molecular_r2.newick',
 'feature_table_decon_mucus_1000.qza',
 'merged_disease_table_r2_names_updated.txt']

## Define paths for all required input files

We'll save the expected filename of each required input file to a variable. This lets us check that all are present before we do any work inside QIIME2.

In [3]:
#Set paths for all required files.
#Currently all are assumed to be directly in the ../input directory
#(if they are somewhere else, just modify the input_directory variable accordingly)
input_directory = abspath("../input")

mapping_file = join(input_directory,"GCMP_EMP_map_r28_no_empty_samples.txt")
taxonomy_file = join(input_directory,"silva_metaxa2_reference_taxonomy.qza") 
tree_file = join(input_directory,"physeq_rooted_tree.qza")
feature_table_all_path = join(input_directory,"feature_table_decon_all_1000.qza")
feature_table_mucus_path = join(input_directory,"feature_table_decon_mucus_1000.qza")
feature_table_tissue_path = join(input_directory,"feature_table_decon_tissue_1000.qza")
feature_table_skeleton_path = join(input_directory,"feature_table_decon_skeleton_1000.qza")

disease_data_path = join(input_directory,"disease_by_genus.csv")

## (Optional) Check that all required files are present

If any are missing, its easier to find out now, before we've spent a long time processing the data.

In [4]:
required_files = [mapping_file,taxonomy_file,tree_file,feature_table_all_path,feature_table_mucus_path,feature_table_tissue_path,\
  feature_table_skeleton_path,disease_data_path]

for required_filepath in required_files:
    
    if not exists(required_filepath):
        raise ValueError(f"Filepath {required_filepath} is needed for this workflow but isn't present. Could it be named something else or in another directory?")

    print(f"Filepath {required_filepath} exists and is accessible...OK")

Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/GCMP_EMP_map_r28_no_empty_samples.txt exists and is accessible...OK
Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/silva_metaxa2_reference_taxonomy.qza exists and is accessible...OK
Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/physeq_rooted_tree.qza exists and is accessible...OK
Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/feature_table_decon_all_1000.qza exists and is accessible...OK
Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/feature_table_decon_mucus_1000.qza exists and is accessible...OK

## Create an output directory if one doesn't exist

In [6]:

output_dir = abspath("../output/")
ft_output_dir = join(output_dir,"feature_tables")

if not exists(ft_output_dir):
    print(f"Output directory {ft_output_dir} does not yet exist, creating it...")
    mkdir(ft_output_dir)
    print("Done.")

## Load feature tables, the coral tree, and metadata into QIIME2 as Artifacts

In [167]:

tree = Artifact.load(tree_file)
taxonomy = Artifact.load(taxonomy_file)
metadata = Metadata.load(mapping_file)
feature_table_decon_all_1000 = Artifact.load(feature_table_all_path)
feature_table_decon_mucus_1000 = Artifact.load(feature_table_mucus_path)
feature_table_decon_tissue_1000 = Artifact.load(feature_table_tissue_path)
feature_table_decon_skeleton_1000 = Artifact.load(feature_table_skeleton_path)

#create a dictionary to hold each feature table so they are easy to access in loops
feature_tables_decon_1000 = {"mucus":feature_table_decon_mucus_1000, "tissue":feature_table_decon_tissue_1000, "skeleton":feature_table_decon_skeleton_1000, "all":feature_table_decon_all_1000}




## Use the Metadata file to find a list of all unique coral genera in the study

In [8]:
# Find unique genus names
df = metadata.to_dataframe()
unique_species_names = list(set(list(df['host_genus'])))
print(f"There are {len(unique_species_names)} unique species names in the dataset:")
print(sorted(unique_species_names))

There are 74 unique species names in the dataset:
['Acanthastrea', 'Acropora', 'Aiptasia', 'Alveopora', 'Astrea', 'Aurelia', 'Caulastraea', 'Coscinaraea', 'Ctenactis', 'Cyphastrea', 'Danafungia', 'Dendrophyllia', 'Diploastrea', 'Diploria', 'Dipsastraea', 'Distichopora', 'Echinophyllia', 'Echinopora', 'Eguchipsammia', 'Entacmaea', 'Favites', 'Fungid', 'Galaxea', 'Gardineroseris', 'Goniastrea', 'Goniopora', 'Heliopora', 'Herpolitha', 'Heteractis', 'Homophyllia', 'Hydnophora', 'Isopora', 'Leptastrea', 'Leptoria', 'Lithophyllon', 'Lobophyllia', 'Lobophytum', 'Macrorhynchia', 'Merulina', 'Millepora', 'Missing: Not collected', 'Mnemiopsis', 'Montastraea', 'Montipora', 'Mycedium', 'Not applicable', 'Orbicella', 'Oxypora', 'Pachyseris', 'Palythoa', 'Pavona', 'Pectinia', 'Physogyra', 'Platygyra', 'Plerogyra', 'Pocillopora', 'Podabacia', 'Porites', 'Psammocora', 'Pseudosiderastrea', 'Rhodactis', 'Sarcophyton', 'Scolymia', 'Seriatopora', 'Siderastrea', 'Sinularia', 'Stephanocoenia', 'Stichodactyl

## Define a function for calculating the most abundant microbe in each host taxon

We want to know, for some level of taxonomic specificity, what the most abundant microbial group was in a particular taxon of hosts. For example, were Proteobacteria the most abundant phylum of microbes in all corals?

**Note: we don't call this function directly - it is instead called by the calculate per-species diversities function

In [98]:
 
#calculate the dominant taxon family
def get_dominant_taxon(feature_table,taxonomy,level=5):
    """Collapse the feature table at the specified level, then return which feature is most abundant and its abundance"""
    try:
        collapse_results = collapse(feature_table,taxonomy,level)
        taxon_table = collapse_results.collapsed_table
        taxon_df = taxon_table.view(pd.DataFrame)
        #Calculate average abundance of each taxon in this species
    except TypeError:
        return None
    taxon_abundance_dict = {taxon_df[col].mean():col for col in list(taxon_df.columns)}
    
    mean_abundance = sorted(taxon_abundance_dict.keys())
    #print(mean_abundance)
    if not mean_abundance:
        return None
    greatest_mean_abundance = mean_abundance[-1]
    most_abundant_taxon = taxon_abundance_dict[greatest_mean_abundance]
    
    return most_abundant_taxon
    
 

## Define a function for calculating alpha diversity within host taxa

We'll keep the specific column for the host taxon generic, so we can calculate per-species alpha diversity using e.g. the 'host_species' column, or per-genus diversity using 'host_genus' as the column.


#### TODO:

Update this approach (it isn't working). Get the dominant taxa into a set. Then just use the above code to collapse the feature table by taxonomy at each level, then filter to just the dominant taxa. Finally, use `merge` to attach these to the results_df

In [151]:
from copy import deepcopy
from qiime2.plugins.taxa.methods import filter_table

def summarize_dominant_taxa(feature_table,\
                            metadata,\
                            taxonomy,\
                            species_column = "host_genus",\
                            compartment_name = 'all',\
                            
                            to_skip = ['none','','Not applicable','Missing: Not collected']\
                            ):
    """Return a DataFrame holding the dominant microbial taxa in each host taxon
    feature_table -- a QIIME2 feature table artifact
    metadata -- a QIIME2 Metadata object
    taxonomy -- a QIIME2 taxonomy object that include the dominant taxa at several 
      levels in the dataframe
    species_column -- the column in metadata by which host taxonomy will be grouped
    compartment_name -- This is specific to the GCMP, and says which 'compartment'
    (mucus,tissue,or skeleton) will be included
   
    """
    #Set up a results_df DataFrame to hold results
    results_columns = [species_column]
    
    
    taxonomy_levels = ['domain','phylum','class','order','family','genus']
    print("Taxonomy levels:",taxonomy_levels)

    #Get unique taxon names from metadata
    metadata_df = metadata.to_dataframe()
    unique_species_names = list(set(list(metadata_df[species_column])))
    
    #Build a dict of microbial taxa dominant in at least 1 coral taxon, keyed by taxonomic level
    dominant_taxa = defaultdict(list)
    
    #Define a dict to cache species tables 
    species_tables = {}
    
    print("*"*40+"Finding Dominant Taxa"+"*"*40)
   

    #Find dominant taxa
    for species in unique_species_names:
        if species in to_skip:
            continue
                 
        print(f"Analyzing {species}...")
        #If taxonomy is provided, summarize the type of microbe with highest average
        #abundance at each taxonomic level
        
        for i, taxon_label in enumerate(taxonomy_levels):
            level = i + 1 #domain is level 1 in QIIME2, not level 0
            #Generate a feature table for this taxon
            where = f"[{species_column}] = '{species}'"
            filter_results = filter_samples(feature_table, metadata = metadata,where = where)
            species_table = filter_results.filtered_table
            species_tables[species]=species_table
            most_abundant_taxon = get_dominant_taxon(species_table,taxonomy,level=level)
            print(f"Most abundant {taxon_label}(level {level}): {most_abundant_taxon}")
            dominant_taxa[level].append(most_abundant_taxon)
     
    #Remove duplicate taxa at each level
    for level, taxon_list in dominant_taxa.items():
        dominant_taxa[level] = sorted(list(set([str(t) for t in taxon_list if t])))
    
    print("Dominant taxa:",dominant_taxa)
    
    for taxon_list in dominant_taxa.values(): 
            taxon_list = [str(t) for t in taxon_list if t]
            results_columns.extend(sorted(list(set(taxon_list))))
    print("Result columns:",results_columns)
    results_df = pd.DataFrame(columns = results_columns)
    results_df = results_df.set_index(species_column)
    
    
    #Now that we have the most abundant taxa added to the dataframe
    #AND have a complete list of the taxa we want from the taxonomy at each level,
    #we can extract relative abundances for each
    
    #Filter the species-level feature tab;e to just the dominant taxa           
    for species in unique_species_names:
        if species in to_skip:
            continue
            
        #Filter the feature table to just our current species
        where = f"[{species_column}] = '{species}'"
        filter_results = filter_samples(feature_table, metadata = metadata,where = where)
        species_table = filter_results.filtered_table
        
        for i, taxon_label in enumerate(taxonomy_levels):
            level = i + 1            
            
            #filter the species table to only the dominant taxa at this level            
            try:
                taxa_to_include = ",".join(dominant_taxa[level])         
                filter_results = filter_table(species_table, taxonomy, include=taxa_to_include)
            except ValueError:
                print("No dominant taxa found in feature table (empty table after filtering)...skipping")
                continue
            dominant_taxa_feature_table = filter_results.filtered_table
                       
            #collapse the species feature table at the current taxonomic level
            collapse_results = collapse(dominant_taxa_feature_table,taxonomy,level)
            taxon_table = collapse_results.collapsed_table
            
            dominant_taxa_abundances = taxon_table.view(pd.DataFrame)
            
            for dominant_taxon in dominant_taxa[level]:
                try:
                    dominant_taxa_mean = dominant_taxa_abundances[dominant_taxon].mean()
                    print(f"{dominant_taxon} mean: {dominant_taxa_mean}")     
                    results_df.loc[species,dominant_taxon] = dominant_taxa_mean
                except KeyError:
                    print("No data for taxon:",dominant_taxon,"...skipping")
                    continue
            #print("Current results df:",results_df)
            
    return results_df

## Generate microbial results .tsv files for each of our species 

In [152]:
relative_abundance_results_dfs = {} 
for compartment,table in sorted(feature_tables_decon_1000.items(),reverse=True):
    result_df = summarize_dominant_taxa(table,metadata,compartment_name = compartment,taxonomy = taxonomy)
    
    species_adiv_path = join(output_dir,f"relative_abundance_trait_table_{compartment}.tsv")
    relative_abundance_results_dfs[compartment]=result_df
    result_df.to_csv(species_adiv_path,sep="\t")
    
    
    
    
    
    
    
    

Taxonomy levels: ['domain', 'phylum', 'class', 'order', 'family', 'genus']
****************************************Finding Dominant Taxa****************************************
Analyzing Lobophyllia...
Most abundant domain(level 1): D_0__Bacteria
Most abundant phylum(level 2): D_0__Bacteria;D_1__Proteobacteria
Most abundant class(level 3): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant order(level 4): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales
Most abundant family(level 5): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae
Most abundant genus(level 6): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae;D_5__Endozoicomonas
Analyzing Alveopora...
Most abundant domain(level 1): D_0__Bacteria
Most abundant phylum(level 2): D_0__Bacteria;D_1__Proteobacteria
Most abundant class(level 3): D_0__Bacteria;D_1__Proteobact

Most abundant order(level 4): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales
Most abundant family(level 5): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Pseudomonadales;D_4__Pseudomonadaceae
Most abundant genus(level 6): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Pseudomonadales;D_4__Pseudomonadaceae;D_5__Pseudomonas
Analyzing Dendrophyllia...
Most abundant domain(level 1): D_0__Bacteria
Most abundant phylum(level 2): D_0__Bacteria;D_1__Proteobacteria
Most abundant class(level 3): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
Most abundant order(level 4): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhodobacterales
Most abundant family(level 5): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhodobacterales;D_4__Rhodobacteraceae
Most abundant genus(level 6): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhodobacterales;D_4__Rhodobacteraceae;_

Most abundant class(level 3): None
Most abundant order(level 4): None
Most abundant family(level 5): None
Most abundant genus(level 6): None
Analyzing Coscinaraea...
Most abundant domain(level 1): D_0__Bacteria
Most abundant phylum(level 2): D_0__Bacteria;D_1__Proteobacteria
Most abundant class(level 3): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant order(level 4): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;__
Most abundant family(level 5): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;__;__
Most abundant genus(level 6): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;__;__;__
Analyzing Goniastrea...
Most abundant domain(level 1): D_0__Bacteria
Most abundant phylum(level 2): D_0__Bacteria;D_1__Proteobacteria
Most abundant class(level 3): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
Most abundant order(level 4): D_0__Bacteria;D_1__Cyanobacteria;D_2__Oxyphotobacteria;D_3__Nostocales
Most abundant family(leve

Most abundant class(level 3): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant order(level 4): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales
Most abundant family(level 5): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae
Most abundant genus(level 6): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae;D_5__Endozoicomonas
Analyzing Entacmaea...
Most abundant domain(level 1): None
Most abundant phylum(level 2): None
Most abundant class(level 3): None
Most abundant order(level 4): None
Most abundant family(level 5): None
Most abundant genus(level 6): None
Analyzing Rhodactis...
Most abundant domain(level 1): D_0__Bacteria
Most abundant phylum(level 2): D_0__Bacteria;D_1__Proteobacteria
Most abundant class(level 3): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
Most abundant order(level 4): D_0__Bacteria;D_

Most abundant genus(level 6): D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Bacteroidales;D_4__Bacteroidetes vadinHA17;D_5__uncultured bacterium
Analyzing Diploastrea...
Most abundant domain(level 1): D_0__Bacteria
Most abundant phylum(level 2): D_0__Bacteria;D_1__Proteobacteria
Most abundant class(level 3): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
Most abundant order(level 4): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales
Most abundant family(level 5): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhodobacterales;D_4__Rhodobacteraceae
Most abundant genus(level 6): D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales;__;__
Analyzing Heliopora...
Most abundant domain(level 1): D_0__Bacteria
Most abundant phylum(level 2): D_0__Bacteria;D_1__Proteobacteria
Most abundant class(level 3): D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant order(level 4): D_0__Bacteria;D_1__

...

D_0__Bacteria mean: 978.0
D_0__Bacteria;D_1__Bacteroidetes mean: 79.0
D_0__Bacteria;D_1__Proteobacteria mean: 558.0
No data for taxon: D_0__Bacteria;__ ...skipping
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia mean: 79.0
D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria mean: 205.0
D_0__Bacteria;D_1__Proteobacteria;D_2__Deltaproteobacteria mean: 10.0
D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria mean: 343.0
No data for taxon: D_0__Bacteria;__;__ ...skipping
No data for taxon: D_0__Archaea;D_1__Thaumarchaeota;D_2__Nitrososphaeria;D_3__Nitrosopumilales ...skipping
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Bacteroidales mean: 4.0
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Cytophagales mean: 7.0
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales mean: 65.0
D_0__Bacteria;D_1__Cyanobacteria;D_2__Oxyphotobacteria;D_3__Synechococcales mean: 125.0
D_0__Bacteria;D_1__Firmicutes;D_2__Clostridia;D_3__Clostridiales mean: 13.0
D

D_0__Bacteria mean: 984.8
D_0__Bacteria;D_1__Bacteroidetes mean: 72.32
D_0__Bacteria;D_1__Proteobacteria mean: 717.72
No data for taxon: D_0__Bacteria;__ ...skipping
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia mean: 69.0
D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria mean: 335.08
D_0__Bacteria;D_1__Proteobacteria;D_2__Deltaproteobacteria mean: 19.8
D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria mean: 360.76
No data for taxon: D_0__Bacteria;__;__ ...skipping
D_0__Archaea;D_1__Thaumarchaeota;D_2__Nitrososphaeria;D_3__Nitrosopumilales mean: 6.28
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Bacteroidales mean: 15.32
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Cytophagales mean: 5.48
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales mean: 25.8
D_0__Bacteria;D_1__Cyanobacteria;D_2__Oxyphotobacteria;D_3__Synechococcales mean: 21.16
D_0__Bacteria;D_1__Firmicutes;D_2__Clostridia;D_3__Clostridiales mean: 21.2
D_0__Bacteria;

D_0__Bacteria mean: 788.0
D_0__Bacteria;D_1__Bacteroidetes mean: 114.0
D_0__Bacteria;D_1__Proteobacteria mean: 304.0
No data for taxon: D_0__Bacteria;__ ...skipping
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia mean: 108.5
D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria mean: 88.0
D_0__Bacteria;D_1__Proteobacteria;D_2__Deltaproteobacteria mean: 158.0
D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria mean: 58.0
No data for taxon: D_0__Bacteria;__;__ ...skipping
D_0__Archaea;D_1__Thaumarchaeota;D_2__Nitrososphaeria;D_3__Nitrosopumilales mean: 140.5
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Bacteroidales mean: 9.5
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Cytophagales mean: 40.0
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales mean: 52.0
D_0__Bacteria;D_1__Cyanobacteria;D_2__Oxyphotobacteria;D_3__Synechococcales mean: 3.5
D_0__Bacteria;D_1__Firmicutes;D_2__Clostridia;D_3__Clostridiales mean: 19.0
D_0__Bacteria;D_1__P

D_0__Bacteria mean: 874.6153846153846
D_0__Bacteria;D_1__Bacteroidetes mean: 81.3076923076923
D_0__Bacteria;D_1__Proteobacteria mean: 579.4615384615385
No data for taxon: D_0__Bacteria;__ ...skipping
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia mean: 74.53846153846153
D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria mean: 230.23076923076923
D_0__Bacteria;D_1__Proteobacteria;D_2__Deltaproteobacteria mean: 31.53846153846154
D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria mean: 316.9230769230769
No data for taxon: D_0__Bacteria;__;__ ...skipping
D_0__Archaea;D_1__Thaumarchaeota;D_2__Nitrososphaeria;D_3__Nitrosopumilales mean: 7.076923076923077
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Bacteroidales mean: 14.23076923076923
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Cytophagales mean: 44.46153846153846
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales mean: 13.307692307692308
D_0__Bacteria;D_1__Cyanobacteria;D_2__Oxyph

D_0__Bacteria mean: 887.5833333333334
D_0__Bacteria;D_1__Bacteroidetes mean: 44.833333333333336
D_0__Bacteria;D_1__Proteobacteria mean: 741.5
No data for taxon: D_0__Bacteria;__ ...skipping
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia mean: 44.166666666666664
D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria mean: 370.5833333333333
D_0__Bacteria;D_1__Proteobacteria;D_2__Deltaproteobacteria mean: 10.416666666666666
D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria mean: 359.1666666666667
No data for taxon: D_0__Bacteria;__;__ ...skipping
D_0__Archaea;D_1__Thaumarchaeota;D_2__Nitrososphaeria;D_3__Nitrosopumilales mean: 1.25
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Bacteroidales mean: 1.3333333333333333
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Cytophagales mean: 20.25
D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales mean: 17.166666666666668
D_0__Bacteria;D_1__Cyanobacteria;D_2__Oxyphotobacteria;D_3__Synechococcales 

In [172]:
#Integrate data into a per species level table
from IPython.core.display import HTML,display

disease_df = pd.read_csv("../input/disease_by_genus.csv")
disease_df = disease_df.set_index('host_genus')
for compartment,results_df in relative_abundance_results_dfs.items():
    results_df = results_df.add_prefix(f"{compartment}_")
    results_df.columns = [c.replace(";","___") for c in results_df.columns]
    results_df.columns = [c.replace(" ","_") for c in results_df.columns]
    results_df.columns = [c.replace("-","_") for c in results_df.columns]
    results_df.index.rename('host_genus')   
    disease_df = pd.merge(disease_df, results_df, how="outer", left_index = True, right_index = True, indicator=False)
disease_df.to_csv("../output/GCMP_trait_table_genus_with_abundances.tsv",sep="\t")


In [168]:
def calculate_per_species_diversities(feature_table,\
                                      metadata,\
                                      species_column = "host_genus",\
                                      compartment_name = 'all',\
                                      taxonomy = None,\
                                      metrics = ['faith_pd','observed_features','gini_index','dominance','simpson_e'],\
                                      to_skip = ['none','','Not applicable','Missing: Not collected']\
                                      ):
    """Return a DataFrame holding diversity results in each species
    
    feature_table -- a QIIME2 feature table artifact
    metadata -- a QIIME2 Metadata object
    species_column -- the column in metadata by which taxonomy will be grouped
    compartment_name -- This is specific to the GCMP, and says which 'compartment'
    (mucus,tissue,or skeleton) will be included
    taxonomy -- a QIIME2 taxonomy object, if provided, will include the dominant taxa at several 
      levels in the dataframe
    """
    #Set up a DataFrame to hold results
    results_columns = [species_column,f"n_samples_{compartment_name}"]
    results_columns.extend([f"{metric}_{compartment_name}" for metric in metrics])
    
    taxonomy_levels = ('domain','phylum','class','order','family','genus')
    taxonomy_labels = [f"most_abundant_{level}_{compartment_name}" for level in taxonomy_levels]
    results_columns.extend(taxonomy_labels)
    
    print("Result columns:",results_columns)
    results_df = pd.DataFrame(columns = results_columns)
    results_df = results_df.set_index(species_column)
    metadata_df = metadata.to_dataframe()
    unique_species_names = list(set(list(metadata_df[species_column])))
    
    #Hold dominant microbial taxa in at least 1 coral taqxon, keyed by taxonomic level
    dominant_taxa = defaultdict(list)
    
    for species in unique_species_names:
        if species in to_skip:
            continue
            
        #Filter the feature table to just our current species
        where = f"[{species_column}] = '{species}'"
        filter_results = filter_samples(feature_table, metadata = metadata,where = where)
        species_table = filter_results.filtered_table
        #We'll use the species table, not the overall feature table from here on down!
        
        print("Analyzing taxon:",species)
        #If taxonomy is provided, summarize the type of microbe with highest average
        #abundance at each taxonomic level
        if taxonomy:
            for i, taxon_label in enumerate(taxonomy_levels):
                level = i + 1 #domain is level 1 in QIIME2, not level 0
                most_abundant_taxon = get_dominant_taxon(species_table,taxonomy,level=level)
                column_label = f"most_abundant_{taxon_label}_{compartment_name}"
                results_df.loc[species,column_label] = most_abundant_taxon
                dominant_taxa[level].append(most_abundant_taxon)
        
        for metric in metrics:
            #print(f"Calculating {metric} for {species}")
            try:
                if metric == "faith_pd":
                    alpha_results = alpha_phylogenetic(species_table,phylogeny=tree,metric=metric)
                else:
                    alpha_results = alpha(species_table, metric = metric)
            except ValueError:
                print(f"Can't calculate {metric} for {species} {compartment_name}...skipping")
                continue
            
            alpha_diversity = alpha_results.alpha_diversity
            species_adiv = alpha_diversity.view(pd.Series)
            
            species_mean = species_adiv.mean()
            results_df.loc[species,f"{metric}_{compartment_name}"] = species_mean
            print(f"{species}\t{metric}\t{compartment_name}\t{round(species_mean,4)}")
            
            
            
        #For last metric only we'll grab n (should be the same for all)
        species_n = len(species_adiv)     
        results_df.loc[species,f"n_samples_{compartment_name}"]=species_n
    return results_df

In [169]:
   
results_dfs = {} 
for compartment,table in sorted(feature_tables_decon_1000.items(),reverse=True):
    print(compartment,table)
    result_df = calculate_per_species_diversities(table,metadata,compartment_name = compartment,taxonomy = taxonomy)
    species_adiv_path = join(output_dir,f"adiv_trait_table_{compartment}.tsv")
    results_dfs[compartment]=result_df
    result_df.to_csv(species_adiv_path,sep="\t")

tissue <artifact: FeatureTable[Frequency] uuid: 1e61a08d-2c29-4dce-9536-6fc182c6f777>
Result columns: ['host_genus', 'n_samples_tissue', 'faith_pd_tissue', 'observed_features_tissue', 'gini_index_tissue', 'dominance_tissue', 'simpson_e_tissue', 'most_abundant_domain_tissue', 'most_abundant_phylum_tissue', 'most_abundant_class_tissue', 'most_abundant_order_tissue', 'most_abundant_family_tissue', 'most_abundant_genus_tissue']
Analyzing taxon: Lobophyllia
Lobophyllia	faith_pd	tissue	73.6568
Lobophyllia	observed_features	tissue	176.1176
Lobophyllia	gini_index	tissue	0.9729
Lobophyllia	dominance	tissue	0.1516
Lobophyllia	simpson_e	tissue	0.0938
Analyzing taxon: Alveopora
Alveopora	faith_pd	tissue	98.9281
Alveopora	observed_features	tissue	241.0
Alveopora	gini_index	tissue	0.8616
Alveopora	dominance	tissue	0.1711
Alveopora	simpson_e	tissue	0.0433
Analyzing taxon: Favites
Favites	faith_pd	tissue	48.1556
Favites	observed_features	tissue	122.6667
Favites	gini_index	tissue	0.944
Favites	dominanc

Analyzing taxon: Palythoa
Palythoa	faith_pd	tissue	104.313
Palythoa	observed_features	tissue	276.0
Palythoa	gini_index	tissue	0.8778
Palythoa	dominance	tissue	0.2571
Palythoa	simpson_e	tissue	0.054
Analyzing taxon: Macrorhynchia
Macrorhynchia	faith_pd	tissue	33.1099
Macrorhynchia	observed_features	tissue	59.0
Macrorhynchia	gini_index	tissue	0.8794
Macrorhynchia	dominance	tissue	0.3209
Macrorhynchia	simpson_e	tissue	0.0528
Analyzing taxon: Tubastraea
Tubastraea	faith_pd	tissue	104.2211
Tubastraea	observed_features	tissue	271.1429
Tubastraea	gini_index	tissue	0.9293
Tubastraea	dominance	tissue	0.1659
Tubastraea	simpson_e	tissue	0.106
Analyzing taxon: Xenia
Xenia	faith_pd	tissue	25.1275
Xenia	observed_features	tissue	42.0
Xenia	gini_index	tissue	0.8296
Xenia	dominance	tissue	0.2084
Xenia	simpson_e	tissue	0.1843
Analyzing taxon: Oxypora
Can't calculate faith_pd for Oxypora tissue...skipping
Can't calculate observed_features for Oxypora tissue...skipping
Can't calculate gini_index for Oxypo

Seriatopora	faith_pd	tissue	33.5398
Seriatopora	observed_features	tissue	48.75
Seriatopora	gini_index	tissue	0.9294
Seriatopora	dominance	tissue	0.177
Seriatopora	simpson_e	tissue	0.1257
skeleton <artifact: FeatureTable[Frequency] uuid: 50609d67-e24a-407c-a0a0-725359be97c9>
Result columns: ['host_genus', 'n_samples_skeleton', 'faith_pd_skeleton', 'observed_features_skeleton', 'gini_index_skeleton', 'dominance_skeleton', 'simpson_e_skeleton', 'most_abundant_domain_skeleton', 'most_abundant_phylum_skeleton', 'most_abundant_class_skeleton', 'most_abundant_order_skeleton', 'most_abundant_family_skeleton', 'most_abundant_genus_skeleton']
Analyzing taxon: Lobophyllia
Lobophyllia	faith_pd	skeleton	68.9772
Lobophyllia	observed_features	skeleton	182.2222
Lobophyllia	gini_index	skeleton	0.9694
Lobophyllia	dominance	skeleton	0.1153
Lobophyllia	simpson_e	skeleton	0.1596
Analyzing taxon: Alveopora
Alveopora	faith_pd	skeleton	71.9609
Alveopora	observed_features	skeleton	155.5
Alveopora	gini_index	sk

Analyzing taxon: Ctenactis
Ctenactis	faith_pd	skeleton	31.5561
Ctenactis	observed_features	skeleton	42.0
Ctenactis	gini_index	skeleton	0.9033
Ctenactis	dominance	skeleton	0.7803
Ctenactis	simpson_e	skeleton	0.0305
Analyzing taxon: Caulastraea
Caulastraea	faith_pd	skeleton	116.6304
Caulastraea	observed_features	skeleton	325.0
Caulastraea	gini_index	skeleton	0.6106
Caulastraea	dominance	skeleton	0.0451
Caulastraea	simpson_e	skeleton	0.0682
Analyzing taxon: Stichodactyla
Can't calculate faith_pd for Stichodactyla skeleton...skipping
Can't calculate observed_features for Stichodactyla skeleton...skipping
Can't calculate gini_index for Stichodactyla skeleton...skipping
Can't calculate dominance for Stichodactyla skeleton...skipping
Can't calculate simpson_e for Stichodactyla skeleton...skipping
Analyzing taxon: Palythoa
Palythoa	faith_pd	skeleton	80.1302
Palythoa	observed_features	skeleton	224.75
Palythoa	gini_index	skeleton	0.8728
Palythoa	dominance	skeleton	0.1783
Palythoa	simpson_e	skele

Dipsastraea	gini_index	skeleton	0.8033
Dipsastraea	dominance	skeleton	0.0459
Dipsastraea	simpson_e	skeleton	0.0986
Analyzing taxon: Platygyra
Platygyra	faith_pd	skeleton	42.4471
Platygyra	observed_features	skeleton	90.2
Platygyra	gini_index	skeleton	0.9329
Platygyra	dominance	skeleton	0.0966
Platygyra	simpson_e	skeleton	0.1986
Analyzing taxon: Echinophyllia
Echinophyllia	faith_pd	skeleton	89.178
Echinophyllia	observed_features	skeleton	232.5
Echinophyllia	gini_index	skeleton	0.8948
Echinophyllia	dominance	skeleton	0.0713
Echinophyllia	simpson_e	skeleton	0.1531
Analyzing taxon: Mnemiopsis
Can't calculate faith_pd for Mnemiopsis skeleton...skipping
Can't calculate observed_features for Mnemiopsis skeleton...skipping
Can't calculate gini_index for Mnemiopsis skeleton...skipping
Can't calculate dominance for Mnemiopsis skeleton...skipping
Can't calculate simpson_e for Mnemiopsis skeleton...skipping
Analyzing taxon: Pachyseris
Pachyseris	faith_pd	skeleton	48.3654
Pachyseris	observed_feature

Millepora	gini_index	mucus	0.9406
Millepora	dominance	mucus	0.129
Millepora	simpson_e	mucus	0.2153
Analyzing taxon: Mycedium
Mycedium	faith_pd	mucus	19.0771
Mycedium	observed_features	mucus	31.0
Mycedium	gini_index	mucus	0.7135
Mycedium	dominance	mucus	0.144
Mycedium	simpson_e	mucus	0.224
Analyzing taxon: Stylaster
Can't calculate faith_pd for Stylaster mucus...skipping
Can't calculate observed_features for Stylaster mucus...skipping
Can't calculate gini_index for Stylaster mucus...skipping
Can't calculate dominance for Stylaster mucus...skipping
Can't calculate simpson_e for Stylaster mucus...skipping
Analyzing taxon: Coscinaraea
Coscinaraea	faith_pd	mucus	60.3817
Coscinaraea	observed_features	mucus	126.0
Coscinaraea	gini_index	mucus	0.8754
Coscinaraea	dominance	mucus	0.0687
Coscinaraea	simpson_e	mucus	0.2168
Analyzing taxon: Goniastrea
Goniastrea	faith_pd	mucus	27.3387
Goniastrea	observed_features	mucus	51.0
Goniastrea	gini_index	mucus	0.8844
Goniastrea	dominance	mucus	0.5482
Goniast

Heliopora	dominance	mucus	0.0463
Heliopora	simpson_e	mucus	0.1241
Analyzing taxon: Dipsastraea
Dipsastraea	faith_pd	mucus	41.7272
Dipsastraea	observed_features	mucus	115.0
Dipsastraea	gini_index	mucus	0.6648
Dipsastraea	dominance	mucus	0.0368
Dipsastraea	simpson_e	mucus	0.2364
Analyzing taxon: Platygyra
Platygyra	faith_pd	mucus	43.3529
Platygyra	observed_features	mucus	92.0
Platygyra	gini_index	mucus	0.9202
Platygyra	dominance	mucus	0.1179
Platygyra	simpson_e	mucus	0.1335
Analyzing taxon: Echinophyllia
Echinophyllia	faith_pd	mucus	52.3579
Echinophyllia	observed_features	mucus	126.5
Echinophyllia	gini_index	mucus	0.9108
Echinophyllia	dominance	mucus	0.4376
Echinophyllia	simpson_e	mucus	0.1505
Analyzing taxon: Mnemiopsis
Can't calculate faith_pd for Mnemiopsis mucus...skipping
Can't calculate observed_features for Mnemiopsis mucus...skipping
Can't calculate gini_index for Mnemiopsis mucus...skipping
Can't calculate dominance for Mnemiopsis mucus...skipping
Can't calculate simpson_e for M

Coscinaraea	gini_index	all	0.9579
Coscinaraea	dominance	all	0.1194
Coscinaraea	simpson_e	all	0.1178
Analyzing taxon: Goniastrea
Goniastrea	faith_pd	all	102.1258
Goniastrea	observed_features	all	274.6429
Goniastrea	gini_index	all	0.9576
Goniastrea	dominance	all	0.1173
Goniastrea	simpson_e	all	0.1542
Analyzing taxon: Ctenactis
Ctenactis	faith_pd	all	43.6192
Ctenactis	observed_features	all	73.6667
Ctenactis	gini_index	all	0.873
Ctenactis	dominance	all	0.3342
Ctenactis	simpson_e	all	0.1389
Analyzing taxon: Caulastraea
Caulastraea	faith_pd	all	86.892
Caulastraea	observed_features	all	221.3333
Caulastraea	gini_index	all	0.8766
Caulastraea	dominance	all	0.1615
Caulastraea	simpson_e	all	0.0524
Analyzing taxon: Stichodactyla
Stichodactyla	faith_pd	all	44.3757
Stichodactyla	observed_features	all	61.0
Stichodactyla	gini_index	all	0.692
Stichodactyla	dominance	all	0.0756
Stichodactyla	simpson_e	all	0.217
Analyzing taxon: Palythoa
Palythoa	faith_pd	all	72.1764
Palythoa	observed_features	all	186.285

Analyzing taxon: Seriatopora
Seriatopora	faith_pd	all	42.3319
Seriatopora	observed_features	all	83.0
Seriatopora	gini_index	all	0.9627
Seriatopora	dominance	all	0.1719
Seriatopora	simpson_e	all	0.1518


## Integrate these new microbial data with existing disease data

In [173]:
#Integrate data into a per species level table
from IPython.core.display import HTML,display

disease_df = pd.read_csv("../output/GCMP_trait_table_genus_with_abundances.tsv",sep="\t")
print(disease_df.columns)
disease_df = disease_df.set_index('host_genus')
for compartment,results_df in results_dfs.items():
    results_df.index.rename('host_genus')    
    disease_df = pd.merge(disease_df, results_df, how="outer", left_index = True, right_index = True, indicator=False)
disease_df.to_csv("../output/GCMP_trait_table_with_abundances_and_adiv.tsv",sep="\t")


Index(['host_genus', 'Unnamed: 0', 'sum_total', 'sum_dis', 'sum_healthy',
       'sum_bbd', 'sum_wd', 'perc_healthy', 'perc_dis', 'perc_bbd',
       ...
       'all_D_0__Bacteria___D_1__Proteobacteria___D_2__Gammaproteobacteria___D_3__Betaproteobacteriales___D_4__Burkholderiaceae___D_5__Aquabacterium',
       'all_D_0__Bacteria___D_1__Proteobacteria___D_2__Gammaproteobacteria___D_3__Betaproteobacteriales__________',
       'all_D_0__Bacteria___D_1__Proteobacteria___D_2__Gammaproteobacteria___D_3__Cellvibrionales___D_4__Spongiibacteraceae___D_5__BD1_7_clade',
       'all_D_0__Bacteria___D_1__Proteobacteria___D_2__Gammaproteobacteria___D_3__Oceanospirillales___D_4__Endozoicomonadaceae___D_5__Endozoicomonas',
       'all_D_0__Bacteria___D_1__Proteobacteria___D_2__Gammaproteobacteria___D_3__Pseudomonadales___D_4__Pseudomonadaceae___D_5__Pseudomonas',
       'all_D_0__Bacteria___D_1__Proteobacteria___D_2__Gammaproteobacteria___D_3__Vibrionales___D_4__Vibrionaceae___D_5__Vibrio',
       'all

# Add in the metadata 

In [185]:
#Integrate data into a per species level table
from IPython.core.display import HTML,display

meta_df = pd.read_csv("../output/GCMP_trait_table_meta.csv")
results_df = pd.read_csv("../output/GCMP_trait_table_with_abundances_and_adiv.tsv",sep="\t")




cols_to_use = meta_df.columns.difference(results_df.columns)
meta_df['host_genus'] = meta_df['host_genus_id']
meta_df = meta_df.set_index('host_genus')
results_df = results_df.set_index('host_genus')

full_df = pd.merge(results_df,meta_df[cols_to_use], how="right", left_index = True,\
  right_index = True, indicator=False)

#Do one-hot encoding of functional group
# Get one hot encoding of columns B
one_hot = pd.get_dummies(full_df['functional_group_sensu_darling'])

# Join the encoded df
full_df = full_df.join(one_hot)

#Final check for columns with characters that will mess up command line calls to R scripts
full_df.columns = [c.replace(";","___") for c in full_df.columns]
full_df.columns = [c.replace(" ","_") for c in full_df.columns]
full_df.columns = [c.replace("-","_") for c in full_df.columns]

#Add special purpose column 'Weedy + Competitive'

full_df.to_csv("../output/GCMP_trait_table_with_abundances_and_adiv_and_metadata.tsv",sep="\t")
print("Final trait table columns:",list(full_df.columns))


Final trait table columns: ['Unnamed:_0', 'sum_total', 'sum_dis', 'sum_healthy', 'sum_bbd', 'sum_wd', 'perc_healthy', 'perc_dis', 'perc_bbd', 'perc_wd', 'tissue_D_0__Bacteria', 'tissue_D_0__Archaea___D_1__Thaumarchaeota', 'tissue_D_0__Bacteria___D_1__Bacteroidetes', 'tissue_D_0__Bacteria___D_1__Proteobacteria', 'tissue_D_0__Bacteria_____', 'tissue_D_0__Archaea___D_1__Thaumarchaeota___D_2__Nitrososphaeria', 'tissue_D_0__Bacteria___D_1__Bacteroidetes___D_2__Bacteroidia', 'tissue_D_0__Bacteria___D_1__Proteobacteria___D_2__Alphaproteobacteria', 'tissue_D_0__Bacteria___D_1__Proteobacteria___D_2__Gammaproteobacteria', 'tissue_D_0__Bacteria__________', 'tissue_D_0__Archaea___D_1__Thaumarchaeota___D_2__Nitrososphaeria___D_3__Nitrosopumilales', 'tissue_D_0__Bacteria___D_1__Bacteroidetes___D_2__Bacteroidia___D_3__Bacteroidales', 'tissue_D_0__Bacteria___D_1__Bacteroidetes___D_2__Bacteroidia___D_3__Flavobacteriales', 'tissue_D_0__Bacteria___D_1__Cyanobacteria___D_2__Oxyphotobacteria___D_3__Nostoca